In [1]:
from csv import reader
import pandas as pd
import numpy as np
import os, glob

folder_pwd = os.getcwd()
os.chdir('..')
os.chdir('../resources/Data - BabyWatcher/1000')
path = os.getcwd()

# Get all csv files
csv_files = glob.glob('*.{}'.format('csv'))
csv_files = [ x for x in csv_files if x != 'pre-processed.csv' ]

os.chdir(folder_pwd)
csv_files

['Batch1.csv',
 'Batch2.csv',
 'Batch3.csv',
 'Batch4.csv',
 'Batch5.csv',
 'Batch6.csv',
 'Batch7.csv',
 'Batch8.csv']

### Read CSVs

In [2]:
def read_csv(file_name):
    df = pd.read_csv(path+'/'+file_name, names=['type','x','y','width', 'height', 'img', 'full_width', 'full_height'])
    df['from_file'] = file_name
    return df

df = pd.DataFrame()
for f in csv_files:
    df = df.append(read_csv(f)).reset_index(drop=True)
    
#df[['from_x','from_y','width', 'height']] = df[['from_x','from_y','width', 'height']].apply(pd.to_numeric, errors='coerce')
df

,type,x,y,width,height,img,full_width,full_height,from_file
0,Head,366,147,124,108,19150_2021-01-25_21.00_20523817.png,800,480,Batch1.csv
1,Arms,264,172,99,49,19150_2021-01-25_21.00_20523817.png,800,480,Batch1.csv
2,Head,352,147,151,125,19158_2021-01-10_19.16_191051467.png,800,480,Batch1.csv
3,Spine,294,236,90,70,19158_2021-01-10_19.16_191051467.png,800,480,Batch1.csv
4,Head,397,136,123,103,19162_2021-01-08_13.46_134330502.png,800,480,Batch1.csv
...,...,...,...,...,...,...,...,...,...
1486,Head,460,133,103,88,21186_2021-05-10_15.09_15847976.png,800,480,Batch8.csv
1487,Spine,347,199,128,43,21186_2021-05-10_15.09_15847976.png,800,480,Batch8.csv
1488,Head,319,193,192,186,21197_2021-04-19_21.36_212921919.png,800,480,Batch8.csv
1489,Head,408,123,118,100,21273_2021-04-21_14.58_145020212.png,800,480,Batch8.csv


### Pre-process

In [3]:
f = ['head', 'arm', 'spine', 'hand', 'leg', 'foot']
df['type'].unique()
for w1 in f:
    for w2 in df['type'].unique().tolist():
        if(sum(c1 != c2 for c1,c2 in zip(w1,w2)) <=1):
            
            df.loc[df['type']==w2, 'type'] = w1
df.loc[df['type'].isin(['feet','Feet']),'type'] = 'foot'
print(df.groupby('type').count()['img'])
print(all(x in f for x in df['type'].unique()))

type
arm        58
foot        4
hand       59
head     1009
leg        13
spine     348
Name: img, dtype: int64
True


In [4]:
# Center images around largest dimension
def center_in_square(row):
    middle = ((2*row['x']+row['width'])/2, (2*row['y']+row['height'])/2)
    size = max(row['width'], row['height'])
    top_left = np.subtract(middle, size/2)
    row['x'] = max(int(top_left[0]), 0)
    row['y'] = max(int(top_left[1]), 0)
    row['size'] = size
    return row

centered = df.apply(center_in_square, axis=1).drop(columns=['width', 'height', 'full_width', 'full_height'])
#centered[(centered['x']>0) & (centered['y']>0)]

grouped_sizes = centered.groupby(['type'], as_index=False).agg(
                      {'size':['mean','std', 'max', 'min']}).set_index('type').droplevel(level=0, axis=1)
grouped_sizes

,mean,std,max,min
type,,,,
arm,97.224138,26.644537,202,55
foot,56.750000,24.757154,84,32
hand,66.101695,25.981887,160,32
head,150.247770,38.899087,321,69
leg,106.846154,33.938292,165,47
spine,144.709770,46.148416,345,55


In [5]:
centered

,type,x,y,img,from_file,size
0,head,366,139,19150_2021-01-25_21.00_20523817.png,Batch1.csv,124
1,arm,264,147,19150_2021-01-25_21.00_20523817.png,Batch1.csv,99
2,head,352,134,19158_2021-01-10_19.16_191051467.png,Batch1.csv,151
3,spine,294,226,19158_2021-01-10_19.16_191051467.png,Batch1.csv,90
4,head,397,126,19162_2021-01-08_13.46_134330502.png,Batch1.csv,123
...,...,...,...,...,...,...
1486,head,460,125,21186_2021-05-10_15.09_15847976.png,Batch8.csv,103
1487,spine,347,156,21186_2021-05-10_15.09_15847976.png,Batch8.csv,128
1488,head,319,190,21197_2021-04-19_21.36_212921919.png,Batch8.csv,192
1489,head,408,114,21273_2021-04-21_14.58_145020212.png,Batch8.csv,118


In [6]:
# import cv2

# df1 = centered.tail(200)

# imgs = dict()
# for im in df['img'].unique():
#     imgs[im] = cv2.imread(path+'/Images/'+im)

# check_imgs = dict()
# def draw(img_n, img_c, test_set, color=(0, 0, 255)):
#     t = test_set[test_set['img']==img_n]
#     def add_rect(row, img_c, color=(0, 0, 255)):
#         start = (int(row['x']), int(row['y']))
#         end = (int(row['x']+row['width']), int(row['y']+row['height']))
#         img_c = cv2.rectangle(img_c, start, end, color, 2)
#         img_c = cv2.putText(img_c, row['type'], (int(row['x']), int(row['y']) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
#     t.apply(add_rect, img_c=img_c, color=color,axis=1)
#     return img_c

# imgs_to_check = df1['img'].unique()
# df1['width'] = df1['size']
# df1['height'] = df1['size']
# for it in imgs_to_check:
#     p_img= draw(it, imgs[it], df)
#     check_imgs[it] = draw(it, p_img, df1, color=(255, 0, 0))

# def slideshow(img_list, title_list):
#     import matplotlib.pyplot as plt
#     import numpy as np
#     from matplotlib.widgets import Slider, Button, RadioButtons
#     %matplotlib notebook
#     fig = plt.figure()
#     viewer = fig.add_subplot(111)

#     fig.show()
#     axcolor = 'lightgoldenrodyellow'
#     axmax  = fig.add_axes([0.15, 0.1, 0.7, 0.02])
#     smax = Slider(axmax, 'idx', 0, len(img_list)-1,valinit=0, valstep=1)

#     def arrow_key_image_control(event):
#         if event.key=='a':
#             update((smax.val-1)%len(img_list))
#         if event.key=='d':
#             update((smax.val+1)%len(img_list))

#     def update(val):
#         smax.val = val
#         viewer.clear() # Clears the previous image
#         viewer.imshow(img_list[int(smax.val)], cmap='gray') # Loads the new image
#         s = str(smax.val+1)+'/'+str(len(img_list))
#         if len(title_list) > smax.val:
#             s+= ' - '+title_list[smax.val]
#         viewer.set_title(s)
#         plt.pause(0.01) # Delay in seconds
#         fig.canvas.draw() # Draws the image to the screen

#     smax.on_changed(update)
#     cid = fig.canvas.mpl_connect('key_press_event', arrow_key_image_control)
#     viewer.imshow(img_list[0], cmap='gray')

# slideshow(list(check_imgs.values()), list(check_imgs.keys()))

### Add none

In [2]:
import random
images = 0
sections_per_image = 4

selected = set()

selected = random.sample(os.listdir(path+'/Images'), min(images, len(os.listdir(path+'/Images'))))

xs = np.random.randint(low=200, high=600, size=(len(selected)*sections_per_image))
ys = np.random.randint(low=200, high=400, size=(len(selected)*sections_per_image))
size = np.random.randint(low=50, high=200, size=(len(selected)*sections_per_image))

data = pd.DataFrame({'type': ['none']*len(selected)*sections_per_image, 'x': xs, 'y': ys,'size':size, 'img': list(selected)*sections_per_image, 'from_file': ['Auto-generated']*images*sections_per_image})
generated = pd.concat([centered, data])
generated

NameError: name 'os' is not defined

In [26]:
generated.to_csv(path+'/pre-processed.csv', index=False)

In [28]:
print(generated.groupby('type').count()['img'])

type
arm        58
foot        4
hand       59
head     1009
leg        13
none     2629
spine     348
Name: img, dtype: int64


In [16]:
generated.sort_values('size')

,type,x,y,img,from_file,size
301,hand,397,105,19350_2021-01-02_22.17_221055362.png,Batch2.csv,32
1026,foot,267,164,20464_2021-03-01_19.14_191416928.png,Batch6.csv,32
1061,hand,398,136,20465_2021-03-25_21.11_20574220.png,Batch6.csv,32
872,hand,329,170,20465_2021-03-23_00.24_02426692.png,Batch5.csv,35
53,hand,419,132,19374_2021-02-13_19.39_193518732.png,Batch1.csv,36
...,...,...,...,...,...,...
547,spine,204,13,19558_2021-01-16_17.50_174712283.png,Batch3.csv,305
413,head,229,193,19891_2021-02-14_20.37_203144986.png,Batch2.csv,311
609,spine,216,23,19990_2021-02-12_22.26_222234136.png,Batch3.csv,321
610,head,190,43,19990_2021-02-17_15.41_15314615.png,Batch3.csv,321
